In [2]:
import pandas as pd
import numpy as np
import json
from tools import calculate_similarity, calculate_performance, calculate_performance_V2, pdf_to_text
import os

In [3]:
json_file = 'articlesJsonTemplate.json'
if os.path.exists(json_file):
    raise FileExistsError("File already found")

df = pd.read_excel('articles.xlsx')

data = []
for index, row in df.iterrows():
    article = {
        "filename": f"Article_{row['Index']}.pdf",
        "title": "" if pd.isna(row['Title']) else row['Title'],
        "link": "" if pd.isna(row['Link']) else row['Link'],
        "extractedText": ""
    }

    data.append(article)

with open('articlesJsonTemplate.json', 'w') as f:
    json.dump(data, f, indent=4)

FileExistsError: File already found

# Initial Model Demo Test

In [5]:
from openai import OpenAI

with open("../openAiToken.txt", "r") as key_file:
    api_key = key_file.read().strip()
os.environ["OPENAI_API_KEY"] = api_key  # Optionally set it as an environment variable
client = OpenAI(api_key=api_key)

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "Write a haiku about recursion in programming."
#         }
#     ]
# )
# print(completion.choices[0].message)


In [22]:
print(completion.choices[0].message)

[
  {
    "yield_strength_value": 262.0,
    "yield_strength_units": "MPa",
    "ultimate_tensile_strength_value": 286.8,
    "ultimate_tensile_strength_units": "MPa",
    "ductility_value": 2.0,
    "ductility_units": "%",
    "hardness_value": 105.0,
    "hardness_units": "HV",
    "modulus_value": 67.5,
    "modulus_units": "GPa",
    "notes": "As-sprayed condition with average measurements from five discreet samples"
  },
  {
    "yield_strength_value": 147.5,
    "yield_strength_units": "MPa",
    "ultimate_tensile_strength_value": 195.1,
    "ultimate_tensile_strength_units": "MPa",
    "ductility_value": 13.0,
    "ductility_units": "%",
    "hardness_value": 65.7,
    "hardness_units": "HV",
    "modulus_value": 64.1,
    "modulus_units": "GPa",
    "notes": "Annealed condition with average values from three different tensile tests"
  },
  {
    "yield_strength_value": 203.4,
    "yield_strength_units": "MPa",
    "ultimate_tensile_strength_value": 216.5,
    "ultimate_tensile_

# Article Demo Test

### Note: This is using GPT-4o-Mini, it is not expected to do well

In [4]:
input_pdf = "documents/Article_8.pdf"
article_text = pdf_to_text(input_pdf)
with open("misc/promptV2.txt", "r") as file:
    prompt_text = file.read()

messages=[ 
    {"role": "system", "content": prompt_text},
    {
        "role": "user",
        "content": article_text
    }
]
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
print(completion.choices[0].message.content)


NameError: name 'client' is not defined

# Initial Model Perforamnce Analysis

In [25]:
completion.choices[0].message

ChatCompletionMessage(content='```json\n[\n    {\n        "yield_strength_value": 190,\n        "yield_strength_units": "MPa",\n        "ultimate_tensile_strength_value": 310,\n        "ultimate_tensile_strength_units": "MPa",\n        "ductility_value": "NA",\n        "ductility_units": "%",\n        "hardness_value": 100,\n        "hardness_units": "HV",\n        "modulus_value": "NA",\n        "modulus_units": "NA",\n        "notes": "As-atomized condition"\n    },\n    {\n        "yield_strength_value": 229,\n        "yield_strength_units": "MPa",\n        "ultimate_tensile_strength_value": "NA",\n        "ultimate_tensile_strength_units": "NA",\n        "ductility_value": "NA",\n        "ductility_units": "NA",\n        "hardness_value": "NA",\n        "hardness_units": "NA",\n        "modulus_value": "NA",\n        "modulus_units": "NA",\n        "notes": "Heat-treated condition after powder annealing"\n    },\n    {\n        "yield_strength_value": 77,\n        "yield_strength_u

In [6]:
# modelName = "gpt-4o-mini"
# modelName = "gpt-4o"
modelName = "o3-mini"
# modelName = "o1-preview"

INPUT_FILE = "misc/promptV2.txt"
# INPUT_FILE = "misc/promptV4.txt"
# INPUT_FILE = 'misc/prompt_figureFlagging_V4.txt'
# INPUT_FILE = 'misc/marcoPrompt1-21-25.txt'
# INPUT_FILE = 'misc/marcoPrompt1-31-25_expanded.txt'
# OUTPUT_FILE = f'modelOutputs/{modelName}.json'
# OUTPUT_FILE = f'modelOutputs/{modelName}_figureFlagging_V4_gt.json'
# OUTPUT_FILE = f'modelOutputs/{modelName}_marcoPrompt1-21-25_gt.json'
# OUTPUT_FILE = f'modelOutputs/{modelName}_marcoPrompt1-31-25_expanded_gt.json'
OUTPUT_FILE = f'modelOutputs/{modelName}_promptv2_gt.json'

with open('articlesJsonTemplate.json') as f:
    articles = json.load(f)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

for article in articles:
    print(article['filename'])
    inGroundTruth = False
    articlePresent = False
    if article['filename'].split('.')[0] in [gt_article for gt_article in ground_truth_articles['Articles']]:
        # print(f"Article {article['filename']} is present in ground truth articles")
        inGroundTruth = True
    else:
        print('Skipping because not in groundtruth')

    if os.path.exists(f'documents/{article["filename"]}'): 
        articlePresent = True
    else:
        print('Skipping Because Article not found')

    if inGroundTruth and articlePresent:
        print(f'{article["filename"]} Ready for Analysis')
        input_pdf = f"documents/{article['filename']}"
        article_text = pdf_to_text(input_pdf)
        with open(INPUT_FILE, "r") as file:
            prompt_text = file.read()
        messages=[ 
            {"role": "user", "content": prompt_text},
            {
                "role": "user",
                "content": article_text
            }
        ]
        completion = client.chat.completions.create(
            model=modelName,
            messages=messages
        )
        extractedVals = completion.choices[0].message.content
        print(extractedVals)
        
        if extractedVals.startswith("```json") and extractedVals.endswith("```"):
            extractedVals = extractedVals[7:-3].strip()


        try:
            article['extractedText'] = json.loads(extractedVals)
        except json.JSONDecodeError as e:
            print(f"Saving as TXT - Error decoding JSON: {e}")
            article['extractedText'] = extractedVals
        except Exception as e:
            print("Another Error Occured")
            raise e
            # article['extractedText'] = extractedVals
        # article['extractedText'] = json.loads(extractedVals)
        with open(OUTPUT_FILE, 'w') as f:
            json.dump(articles, f, indent=4)
    

Article_1.pdf
Article_1.pdf Ready for Analysis
[
  {
    "yield_strength_value": "NA",
    "yield_strength_units": "NA",
    "ultimate_tensile_strength_value": "NA",
    "ultimate_tensile_strength_units": "NA",
    "ductility_value": "NA",
    "ductility_units": "NA",
    "hardness_value": "NA",
    "hardness_units": "NA",
    "modulus_value": 61.4,
    "modulus_units": "GPa",
    "notes": "For cold‐sprayed Al 6061 deposits, only the elastic modulus was explicitly reported (61.4 ± 0.3 GPa in the in‐plane L/T orientation). Yield strength, tensile (ultimate) strength, ductility, and hardness were mentioned qualitatively (or as percentage differences relative to wrought material) but absolute numeric values were not provided."
  }
]
Article_2.pdf
Skipping Because Article not found
Article_3.pdf
Article_3.pdf Ready for Analysis
[
  {
    "yield_strength_value": 151,
    "yield_strength_units": "MPa",
    "ultimate_tensile_strength_value": 311,
    "ultimate_tensile_strength_units": "MPa",


In [8]:
import os

modelName = "o1-mini_marcoPrompt1-31-25_expanded_gt"

with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))



Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'gas_type': 'Helium', 'gas_speed': 1110, 'gas_speed_units': 'm/s', 'gas_pressure': 3.5, 'gas_pressure_units': 'MPa', 'gas_temperature': 400, 'gas_temperature_units'

In [5]:
import os

modelName = "o1-mini_marcoPrompt1-31-25_expanded_gt"

with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))



Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'gas_type': 'Helium', 'gas_speed': 1110, 'gas_speed_units': 'm/s', 'gas_pressure': 3.5, 'gas_pressure_units': 'MPa', 'gas_temperature': 400, 'gas_temperature_units'

In [8]:
import os

modelName = "o1-mini"

with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))

Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'Mean elastic modulus for L/T cold sprayed Al 6061 specimens'}]
Field RMSE
{'yield_strength_value': np.float64(1.0), 'ultimate_tensile_strength_value': 0, 

In [10]:
with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))

Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'Mean elastic modulus for L/T cold sprayed Al 6061 specimens'}]
Field RMSE
{'yield_strength_value': np.float64(1.0), 'ultimate_tensile_strength_value': 0, 

In [61]:
with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))

Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 'NA', 'modulus_units': 'NA', 'notes': 'NA'}]
Field RMSE
{'yield_strength_value': np.float64(1.0), 'ultimate_tensile_strength_value': 0, 'ductility_value': np.float64(1.0), 'hardness_value': 0, '

In [59]:
modelName = 'o1-mini'

with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))

Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'Mean elastic modulus for L/T cold sprayed Al 6061 specimens'}]
Field RMSE
{'yield_strength_value': np.float64(1.0), 'ultimate_tensile_strength_value': 0, 

In [58]:
with open(f'modelOutputs/{modelName}.json', 'r') as file:
    data = json.load(file)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

losses = []

for row in data:
    rowName = row['filename'].split('.')[0]
    rowResults = row.get('extractedText')
    if rowResults != "":
        # rowResults_json = json.dumps(rowResults, indent=2)
        ground_truth_article = ground_truth_articles.get("Articles", {}).get(rowName)
        
        if ground_truth_article is not None:

            print(f"Ground Truth for {rowName}:")
            print(type(ground_truth_article))
            print(ground_truth_article)
            print('-'*30)
            print(f"Extracted for {rowName}:")
            print(type(rowResults))
            print(rowResults)
            results = calculate_performance(ground_truth_article, rowResults)
            losses.append(results['Overall Loss'])
            for i in results:
                print(i)
                print(results[i])
                print('-'*30)
            
        else:
            print(f"No ground truth found for '{rowName}'.\n")
    print('\n'*3)
print('Average Loss: ', np.mean(losses))

Ground Truth for Article_1:
<class 'list'>
[{'yield_strength_value': 260, 'yield_strength_units': 'MPa', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 2.6, 'ductility_units': '%', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'He-sprayed Al6061 sample (Al-2-1) with improved particle impact velocity; shows significant tensile strength increase and ductility.'}]
------------------------------
Extracted for Article_1:
<class 'list'>
[{'yield_strength_value': 'NA', 'yield_strength_units': 'NA', 'ultimate_tensile_strength_value': 'NA', 'ultimate_tensile_strength_units': 'NA', 'ductility_value': 'NA', 'ductility_units': 'NA', 'hardness_value': 'NA', 'hardness_units': 'NA', 'modulus_value': 61.4, 'modulus_units': 'GPa', 'notes': 'Elastic modulus measured for L/T cold-sprayed Al 6061 specimens, sprayed using helium as the accelerating gas.'}]
Field RMSE
{'yield_strength_value': np.f

In [ ]:
# modelName = "gpt-4o-mini"
# modelName = "gpt-4o"
modelName = "o1-mini"
# modelName = "o1-preview"

INPUT_FILE = 'misc/marcoPrompt1-31-25_expanded.txt'
# INPUT_FILE = 'misc/prompt_figureFlagging_V1.txt'
# OUTPUT_FILE = f'modelOutputs/{modelName}_figureFlagging_V1_nonGt.json'
OUTPUT_FILE = f'modelOutputs/{modelName}_marcoPrompt1-31-25_expanded.json'

with open('articlesJsonTemplate.json') as f:
    articles = json.load(f)

with open('misc/groundTruthArticles.json') as f:
    ground_truth_articles = json.load(f)

for article in articles:
    print(article['filename'])
    if '56' not in article['filename'] and '89' not in article['filename']:
        inGroundTruth = False
        articlePresent = False
        if article['filename'].split('.')[0] in [gt_article for gt_article in ground_truth_articles['Articles']]:
            # print(f"Article {article['filename']} is present in ground truth articles")
            inGroundTruth = True
            print('Skipping because in groundtruth')
        else:
            # print('Skipping because not in groundtruth')
            pass

        if os.path.exists(f'documents/{article["filename"]}'): 
            articlePresent = True
        else:
            print('Skipping Because Article not found')

        if articlePresent and not inGroundTruth:
            print(f'{article["filename"]} Ready for Analysis')
            input_pdf = f"documents/{article['filename']}"
            article_text = pdf_to_text(input_pdf)
            with open(INPUT_FILE, "r") as file:
                prompt_text = file.read()
            messages=[ 
                {"role": "user", "content": prompt_text},
                {
                    "role": "user",
                    "content": article_text
                }
            ]
            completion = client.chat.completions.create(
                model=modelName,
                messages=messages
            )
            extractedVals = completion.choices[0].message.content
            print(extractedVals)
            
            if extractedVals.startswith("```json") and extractedVals.endswith("```"):
                extractedVals = extractedVals[7:-3].strip()


            try:
                article['extractedText'] = json.loads(extractedVals)
            except json.JSONDecodeError as e:
                print(f"Saving as TXT - Error decoding JSON: {e}")
                article['extractedText'] = extractedVals
            except Exception as e:
                print("Another Error Occured")
                raise e
                # article['extractedText'] = extractedVals
            # article['extractedText'] = json.loads(extractedVals)
            with open(OUTPUT_FILE, 'w') as f:
                json.dump(articles, f, indent=4)
        

Article_1.pdf
Skipping because in groundtruth
Article_2.pdf
Skipping because in groundtruth
Skipping Because Article not found
Article_3.pdf
Skipping because in groundtruth
Article_4.pdf
Skipping because in groundtruth
Article_5.pdf
Skipping because in groundtruth
Article_6.pdf
Article_6.pdf Ready for Analysis
```json
[
  {
    "yield_strength_value": 385.33,
    "yield_strength_units": "MPa",
    "ultimate_tensile_strength_value": "NA",
    "ultimate_tensile_strength_units": "NA",
    "ductility_value": "NA",
    "ductility_units": "NA",
    "hardness_value": "NA",
    "hardness_units": "NA",
    "modulus_value": "NA",
    "modulus_units": "NA",
    "gas_type": "NA",
    "gas_speed": "NA",
    "gas_speed_units": "NA",
    "gas_pressure": "NA",
    "gas_pressure_units": "NA",
    "gas_temperature": "NA",
    "gas_temperature_units": "NA",
    "standoff_distance": "NA",
    "standoff_distance_units": "NA",
    "spray_gun_speed": "NA",
    "spray_gun_speed_units": "NA",
    "powder_heat_

In [15]:
import json

with open('misc/groundTruthArticles.json', 'r') as f1, open('modelOutputs/o1-mini_v2_nonGT_12_29.json', 'r') as f2:
    ground_truth_articles = json.load(f1)
    non_gt = json.load(f2)

for article in non_gt:
    print(article['filename'])
    inGroundTruth = False
    articlePresent = False
    if article['filename'].split('.')[0] in [gt_article for gt_article in ground_truth_articles['Articles']]:
        article['extractedText'] = ground_truth_articles['Articles'][article['filename'].split('.')[0]]
        article['ground_truth'] = True
    else:
        article['ground_truth'] = False

    with open('modelOutputs/o1-mini_v2_nonGT_12_29_merged.json', 'w') as f:
        json.dump(non_gt, f, indent=4)

Article_1.pdf
Article_2.pdf
Article_3.pdf
Article_4.pdf
Article_5.pdf
Article_6.pdf
Article_7.pdf
Article_8.pdf
Article_9.pdf
Article_10.pdf
Article_11.pdf
Article_12.pdf
Article_13.pdf
Article_14.pdf
Article_15.pdf
Article_16.pdf
Article_17.pdf
Article_18.pdf
Article_19.pdf
Article_20.pdf
Article_21.pdf
Article_22.pdf
Article_23.pdf
Article_24.pdf
Article_25.pdf
Article_26.pdf
Article_27.pdf
Article_28.pdf
Article_29.pdf
Article_30.pdf
Article_31.pdf
Article_32.pdf
Article_33.pdf
Article_34.pdf
Article_35.pdf
Article_36.pdf
Article_37.pdf
Article_38.pdf
Article_39.pdf
Article_40.pdf
Article_41.pdf
Article_42.pdf
Article_43.pdf
Article_44.pdf
Article_45.pdf
Article_46.pdf
Article_47.pdf
Article_48.pdf
Article_49.pdf
Article_50.pdf
Article_51.pdf
Article_52.pdf
Article_53.pdf
Article_54.pdf
Article_55.pdf
Article_56.pdf
Article_57.pdf
Article_58.pdf
Article_59.pdf
Article_60.pdf
Article_61.pdf
Article_62.pdf
Article_63.pdf
Article_64.pdf
Article_65.pdf
Article_66.pdf
Article_67.pdf
Arti

In [21]:
import json

# Load JSON files
with open('modelOutputs/o1-mini_marcoPrompt1-31-25_expanded_gt.json', 'r') as f1, open('modelOutputs/o1-mini_marcoPrompt1-31-25_expanded.json', 'r') as f2:
    ground_truth_articles = json.load(f1)  # Ground truth articles (list or dict)
    non_gt = json.load(f2)  # List of non-ground-truth articles

# Determine the structure of `ground_truth_articles`
if isinstance(ground_truth_articles, dict) and 'Articles' in ground_truth_articles:
    # If 'Articles' key exists, extract the actual list
    ground_truth_articles = ground_truth_articles['Articles']

if not isinstance(ground_truth_articles, list):
    raise ValueError("Expected 'Articles' to be a list in the ground truth JSON.")

# Convert ground truth articles into a dictionary for easy lookup
gt_dict = {}
for article in ground_truth_articles:
    if 'filename' in article:
        base_filename = article['filename'].split('.')[0]
        gt_dict[base_filename] = article.get('extractedText', "")

# Process the non-ground-truth articles
for article in non_gt:
    print(article['filename'])
    
    base_filename = article['filename'].split('.')[0]  # Extract base filename

    if base_filename in gt_dict:
        gt_text = gt_dict[base_filename]  # Ground truth extractedText
        non_gt_text = article.get('extractedText', "")  # Non-ground-truth extractedText
        
        # If ground truth text is empty ("" or empty list), keep non-ground-truth text
        if not gt_text:
            article['extractedText'] = non_gt_text
        else:
            article['extractedText'] = gt_text
        
        article['ground_truth'] = True
    else:
        article['ground_truth'] = False

# Save the merged results
with open('modelOutputs/o1-mini_marcoPrompt1-31-25_expanded_merged.json', 'w') as f:
    json.dump(non_gt, f, indent=4)

print("Merged JSON file saved successfully.")


Article_1.pdf
Article_2.pdf
Article_3.pdf
Article_4.pdf
Article_5.pdf
Article_6.pdf
Article_7.pdf
Article_8.pdf
Article_9.pdf
Article_10.pdf
Article_11.pdf
Article_12.pdf
Article_13.pdf
Article_14.pdf
Article_15.pdf
Article_16.pdf
Article_17.pdf
Article_18.pdf
Article_19.pdf
Article_20.pdf
Article_21.pdf
Article_22.pdf
Article_23.pdf
Article_24.pdf
Article_25.pdf
Article_26.pdf
Article_27.pdf
Article_28.pdf
Article_29.pdf
Article_30.pdf
Article_31.pdf
Article_32.pdf
Article_33.pdf
Article_34.pdf
Article_35.pdf
Article_36.pdf
Article_37.pdf
Article_38.pdf
Article_39.pdf
Article_40.pdf
Article_41.pdf
Article_42.pdf
Article_43.pdf
Article_44.pdf
Article_45.pdf
Article_46.pdf
Article_47.pdf
Article_48.pdf
Article_49.pdf
Article_50.pdf
Article_51.pdf
Article_52.pdf
Article_53.pdf
Article_54.pdf
Article_55.pdf
Article_56.pdf
Article_57.pdf
Article_58.pdf
Article_59.pdf
Article_60.pdf
Article_61.pdf
Article_62.pdf
Article_63.pdf
Article_64.pdf
Article_65.pdf
Article_66.pdf
Article_67.pdf
Arti